# Formularios de Django

Vamos crear una forma agradable de agregar y editar posts en el blog. El admin de Django está bien, pero es bastante difícil de personalizar y hacerlo bonito. Con forms tendremos un poder absoluto sobre nuestra interfaz; ¡podemos hacer casi cualquier cosa que podamos imaginar!

Lo bueno de los formularios de Django es que podemos definirlos desde cero o crear un `ModelForm`, el cual guardará el resultado del formulario en el modelo.

Esto es exactamente lo que queremos hacer: crearemos un formulario para nuestro modelo `Post`.

Como cada parte importante de Django, los formularios tienen su propio archivo: `forms.py`.

Necesitamos crear un archivo con este nombre en el directorio `blog`.

```bash
blog
   └── forms.py
```

Lo abrimos en el editor de código y teclea lo siguiente:

```python
from django import forms

from .models import Post

class PostForm(forms.ModelForm):

    class Meta:
        model = Post
        fields = ('title', 'text',)
```

Lo primero, necesitamos importar Django forms (`from django import forms`) y nuestro modelo `Post` (`from .models import Post`).

`PostForm`, como probablemente sospechas, es el nombre de nuestro formulario. Necesitamos decirle a Django que este formulario es un `ModelForm` (así Django hará algo de magia por nosotros) - `forms.ModelForm` es responsable de ello.

Luego, tenemos `class Meta`, donde le decimos a Django que modelo debe ser utilizado para crear este formulario (`model = Post`).

Finalmente, podemos decir que campo(s) deberían estar en nuestro formulario. En este escenario sólo queremos `title` y `text` para ser mostrados - `author` será la persona que esta conectada y `created_date` se definirá automáticamente cuando creemos un post (es decir, en el código).

¡Y eso es todo! Todo lo que necesitamos hacer ahora es usar el formulario en una view y mostrarla en una plantilla.

Una vez más vamos a crear: un enlace a la página, una dirección URL, una vista y una plantilla.

# Enlace a una página con el formulario

Ahora toca abrir el fichero `blog_curso/templates/blog_curso/post_list.html` en el editor. Vamos a añadir un enlace en el div llamado page-header:

```html
<a href="{% url 'posts_new' %}" class="button"><span class="glyphicon glyphicon-plus"></span> Agregar Post</a>
```

Ten en cuenta que queremos llamar a nuestra nueva vista post_new. La clase `"glyphicon glyphicon-plus"` es proporcionada por el tema de bootstrap que estamos utilizando, y nos mostrara un signo de suma.

# URL

Abrimos `blog/urls.py` en el editor para añadir una línea:

```python
path('posts/new', views.post_new, name='post_new'),
```

Y el código final tendrá este aspecto:

```python
from django.contrib import admin
from django.urls import path

from blog_curso import views

urlpatterns = [
    path('admin/', admin.site.urls),
    path('posts/', views.posts),
    path('posts/new', views.post_new, name='posts_new'),
]
```

# Vista post_new

Ahora abre el fichero `blog/views.py` en el editor y añade estas líneas con el resto de imports from:

```python
from .forms import PostForm
```

Y ahora nuestra vista:

```python
from django.views.generic import FormView

from .forms import PostForm


class PostFormView(FormView):
    form_class = PostForm
    success_url = 'posts_list'
    template_name = "blog_curso/post_new.html"

    def form_valid(self, form):
        post = form.save(commit=False)
        post.author = self.request.user
        post.published_date = timezone.now()
        post.save()
        return super(PostFormView, self).form_valid(form)

post_new = PostFormView.as_view()
```